# Xifrat i probabilitats

En aquest notebook volem abordar una tasca molt concreta: descodificar codis secrets. 

Ho farem amb el més simple dels codis, un xifrat de rotació, de vegades anomenat xifratge de majúscules o xifratge Cèsar (perquè es tractava d’última generació de criptografia al 100 aC). 



In [1]:
import re
import math
import string
import random
from collections import Counter
from math import log10
from __future__ import division
from matplotlib.pyplot import yscale, xscale, title, plot
import requests

alphabet = 'abcdefghijklmnopqrstuvwxyz'

Per fer-ho ens basarem en les probabilitat de les paraules que poden apareixer en el text. 

Començarem llegint un text molt gran per estimar-les.

In [2]:
r = requests.get('https://raw.githubusercontent.com/norvig/pytudes/master/data/text/big.txt')
r.text
TEXT = r.text
# Llegim tots els caràcters d'un text força gran i els guardem a la variable TEXT
print('Tenim un text amb',len(TEXT),'caracters')

Tenim un text amb 6488666 caracters


Anem ara a veure quants *tokens* (o paraules úniques) hi ha i quantes vegades surt cada paraula.

In [3]:
def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower()) 

# Words serà el conjunt de paraules, totes passades a minuscules per normalitzar-ho tot al mateix format
WORDS = tokens(TEXT)
print('Tenim un text amb',len(WORDS),'paraules.')

# Counts guardarà el nombre de vegades que hi ha una certa paraula. Per tant, la longitud del comptador
# ens dóna precisament el nombre de paraules diferents
COUNTS = Counter(WORDS)
print('Tenim un text amb',len(COUNTS),'paraules diferents.')

# Vegades que surten les 10 paraules més freqüents
print('Les 10 paraules més freqüents amb la seva freqüència:\n', COUNTS.most_common(10))

# Vegades que surten les paraules d'un text
for w in tokens('the rarest and more hermitian words'):
    print('La paraula \"', w, '\" surt ', COUNTS[w], ' vegades.')

Tenim un text amb 1105285 paraules.
Tenim un text amb 29157 paraules diferents.
Les 10 paraules més freqüents amb la seva freqüència:
 [('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
La paraula " the " surt  80030  vegades.
La paraula " rarest " surt  0  vegades.
La paraula " and " surt  38313  vegades.
La paraula " more " surt  1997  vegades.
La paraula " hermitian " surt  0  vegades.
La paraula " words " surt  460  vegades.


Anem ara a calcular la distribució de probabilitats de les paraules i d'una frase formada per aquestes paraules.

In [4]:
# Aquesta funció assigna la probabilitat de cada paraula de ser al corpus com la fracció de vegades que hi apareix.
# Si una paraula no hi apareix, se li assigna una probabilitat de 1/(N+1), ja que hi ha paraules que pertanyen al
# corpus amb una freqüència de 1. Per tant, tota paraula que no apareixi se li assigna una probabilitat més petita
# que qualsevol que si que hi apareix però no és pas zero (ja que hi ha paraules que existeixen i no s'han trobat
# al corpus, com per exemple "hermitian").

def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N if counter[x] > 0 else 1./(N + 1)

# Pword assignarà a cada paraula la seva probabilitat
Pword = pdist(COUNTS)

# Imprimim les probabilitats d'unes quantes paraules
for w in tokens('"The" is most common word in English, hermitian is not.'):
    print('La probabilitat de \"', w, '\" és', Pword(w))
    
# Probabilitat d'una frase si tenim en compte que cada paraula és independent de les altres.
def Pwords(words):
    "Probability of a sentence, assuming each word is independent of others."
    return product(Pword(w) for w in words)

# Funció producte (ja que no utilitzem llibreria numpy)
def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

print("Probabilitat d'una frase:",Pwords(['large', 'and', 'insignificant']))

La probabilitat de " the " és 0.07240666434449033
La probabilitat de " is " és 0.008842968103249388
La probabilitat de " most " és 0.0008215075749693518
La probabilitat de " common " és 0.0002596615352601365
La probabilitat de " word " és 0.0002696137195383996
La probabilitat de " in " és 0.019949605757790978
La probabilitat de " english " és 0.00019090098933759167
La probabilitat de " hermitian " és 9.047432067356323e-07
La probabilitat de " is " és 0.008842968103249388
La probabilitat de " not " és 0.005994833911615556
Probabilitat d'una frase: 4.111418791681202e-10


Per últim, anem a definir l'esquema de codificació.

In [5]:
# Retorna el mateix text primer en majuscules i seguidament en minúscules: upperlower('hola') -> HOLAhola
def upperlower(text): return text.upper() + text.lower()  

# Xifra un missatge rotant n vegades l'alfabet. Si n=0, el missatge i el xifrat coincideixen.
def rot(msg, n=13): 
    "Encode a message with a rotation (Caesar) cipher." 
    return encode(msg, alphabet[n:]+alphabet[:n])

def encode(msg, key): 
    "Encode a message with a substitution cipher." 
    table = str.maketrans(upperlower(alphabet), upperlower(key))
    return msg.translate(table) 

# Exemple de missatge xifrat
msg = 'Who knows the answer this time?'
secret = rot(msg, 1)

print('Text:       ',msg)
print('Text xifrat:',secret)

Text:        Who knows the answer this time?
Text xifrat: Xip lopxt uif botxfs uijt ujnf?


## Exercici: 

Completa el codi de la funció 'decode' seguint les següents idees:

+ El xifrat de la frase només pot tenir tantes claus diferents com lletres diferents hi ha.
+ Si provem tots els possibles desxifrats possibles, el correcte és el que dona la seqüencia de paraules més probable.

Tingues en compte també que potser cal modificar la funció `Pword` de manera que cap paraula tingui probabilitat 0.

In [6]:
# Funció que calcula el logaritme decimal de la probabilitat d'una frase.
# Ho implemento amb logaritmes ja que és més fàcil fer sumes o restes que productes o divisions,
# i no tindré problemes d'underflow ni perdré informació perquè un número pugui ser massa proper a zero.
def log_Pwords(words):
    "Log-probability of a sentence, assuming each word is independent of others."
    return sum(log10(Pword(w)) for w in words)


def decode(secret):
    # Començo amb el màxim de la probabilitat que pot tenir una frase inicialitzat a -inf.
    max_prob = float('-inf')
    decoded_final = secret
    for i in range(len(alphabet)):
        # Per cada rotació possible (n'hi ha una per lletra de l'alfabet, ja que sempre l'alfabet estarà ordenat)
        # provo de desxifrar el missatge rotant-lo a la inversa
        decoded = rot(secret, -i)
        # Calculo la probabilitat de la frase desxifrada.
        log_prob = log_Pwords(tokens(decoded))
        # Si la frase desxifrada és la més probable fins al moment, em guardo la probabilitat actual
        # i actualitzo quina ha estat la frase més probable fins aleshores.
        if log_prob > max_prob:
            max_prob = log_prob
            decoded_final = decoded
        # Finalment, retorno la frase més probable juntament amb la seva probabilitat
    return decoded_final, max_prob
# Noteu que quan parlo de probabilitat en la funció anterior sempre estic fent referència al logaritme de
# la probabilitat.

#Proves amb diversos missatges i diverses rotacions:

msg = 'Who knows the answer this time?'
secret = rot(msg, 12)

print(secret)
print(decode(secret), '\n')


msg = 'Are you ok?'
secret = rot(msg, 17)

print(secret)
print(decode(secret), '\n')


msg = 'Mr. and Mrs. Dursley of number four, Privet Drive, were proud to say \
that they were perfectly normal, thank you very much.'
secret = rot(msg, 9)

print(secret)
print(decode(secret), '\n')

msg = 'His hand closed automatically around the fake Horcrux, but in spite of everything, \
in spite of the dark and twisting path he saw stretching ahead for himself, in spite of the \
final meeting with Voldemort he knew must come, whether in a month, in a year, or in ten, he \
felt his heart lift at the thought that there was still one last golden day of peace left to \
enjoy with Ron and Hermione.'
secret = rot(msg, 19)

print(secret)
print(decode(secret), '\n')

Ita wzaie ftq mzeiqd ftue fuyq?
('Who knows the answer this time?', -16.759684660297722) 

Riv pfl fb?
('Are you ok?', -10.820625362239902) 

Va. jwm Vab. Mdabunh xo wdvkna oxda, Yarenc Maren, fnan yaxdm cx bjh cqjc cqnh fnan ynaonlcuh wxavju, cqjwt hxd enah vdlq.
('Mr. and Mrs. Dursley of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.', -73.21649994483423) 

Abl atgw vehlxw tnmhftmbvteer tkhngw max ytdx Ahkvknq, unm bg libmx hy xoxkrmabgz, bg libmx hy max wtkd tgw mpblmbgz itma ax ltp lmkxmvabgz taxtw yhk abflxey, bg libmx hy max ybgte fxxmbgz pbma Ohewxfhkm ax dgxp fnlm vhfx, paxmaxk bg t fhgma, bg t rxtk, hk bg mxg, ax yxem abl axtkm ebym tm max mahnzam matm maxkx ptl lmbee hgx etlm zhewxg wtr hy ixtvx exym mh xgchr pbma Khg tgw Axkfbhgx.
('His hand closed automatically around the fake Horcrux, but in spite of everything, in spite of the dark and twisting path he saw stretching ahead for himself, in spite of the final meeting with 